# Direct and inverse geometry of 3d robots
In the previous class, we worked with 2d robot. Let's now move to 3D cases, with a real manipulator robot. To begin, let's discuss quickly 3D rotations and placements.
## Rotations and placement, SO(3) and SE(3)
A placement is composed of a rotation and a translation. They are represented in Pinocchio as SE3 objects, containing a 3x3 unit matrix and a 3x1 vector.

In [54]:
import pinocchio
from pinocchio import SE3,Quaternion
import numpy as np
from numpy.linalg import norm
M = SE3.Random()
R = M.rotation
p = M.translation
print('R =\n'+str(R))
print('p =\n'+str(p.T))

R =
[[-0.56691857  0.31312074  0.76194405]
 [-0.34878719  0.74670674 -0.56637138]
 [-0.74629138 -0.58684278 -0.31410941]]
p =
[[ 0.05348996  0.53982767 -0.19954276]]


A rotation is simply a 3x3 matrix. It has a unit norm:

In [55]:
print(R*R.T)

[[ 1.00000000e+00 -5.12100243e-17 -9.89194634e-17]
 [-5.12100243e-17  1.00000000e+00 -1.90348506e-17]
 [-9.89194634e-17 -1.90348506e-17  1.00000000e+00]]


It can be equivalently represented by a quaternion. Here we have made the choice to create a specific class for quaternions (i.e. they are not vectors, and can be e.g. multiplied), but you can get the 4 coefficients with the adequate method. Note that the corresponding vector is also of norm 1.

In [56]:
quat = Quaternion(R)
print(norm(quat.coeffs()))

1.0


Angle-axis representation are also implemented in the class AngleAxis. In case of errors at AngleAxis importation, just modify your file /opt/openrobots/lib/python2.7/site-packages/pinocchio/__init__.py to comment the last line (sorry for the bug).

In [57]:
from pinocchio import AngleAxis
utheta = AngleAxis(quat)
print(utheta.angle, utheta.axis.T)

(2.1738505809056377, matrix([[-0.01242789,  0.91562788, -0.40183471]]))


You can display rotation in Gepetto Viewer (remember to first run gepetto-gui from command line to start the viewer).

In [58]:
import gviewserver
gv = gviewserver.GepettoViewerServer()
gv.addBox ('world/box',    .1,.2,.3,    [1 ,0 ,0,1])
gv.applyConfiguration('world/box',[.1,.2,.3]+quat.coeffs().T.tolist()[0])
gv.refresh()

## Quaternion you said?
Quaternions are "complex of complex", introduced form complex as complex are from reals. Let's try to understand what they contains in practice. Quaternions are of the form w+xi+yj+zk, with w,x,y,z real values, and i,j,k the 3 imaginary numbers. We store them as 4-d vectors, with the real part first: quat = [x,y,z,w]. We can interprete w as encoding the cosinus of the rotation angle. Let's see that.

In [59]:
from numpy import arccos
print(arccos(quat[3]))
print(AngleAxis(quat).angle)

1.0869252904528188
2.17385058091


Indeed, w = cos(theta/2). Why divided by two? For that, let's see how the quaternion can be used to represent a rotation. We can encode a 3D vector in the imaginary part of a quaternion.

In [60]:
from pinocchio.utils import rand
p = rand(3)
qp = Quaternion(0.,p[0,0],p[1,0],p[2,0])
print(qp)

(x,y,z,w) =   0.506737   0.748602 0.00599194          0



The real product extends over quaternions, so let's try to multiply quat with p:

In [61]:
print(quat*qp)

(x,y,z,w) =  0.506876  0.168074 -0.416165 -0.599047



Well that's not a pure imaginary quaternion anymore. And the imaginary part does not contains somethig that looks like the rotated point:

In [62]:
print((quat.matrix()*p).T)

[[-0.04830993  0.3788496  -0.81936727]]


The pure quaternion is obtained by multiplying again on the left by the conjugate (w,-x,-y,-z).

In [63]:
print(quat*qp*quat.conjugate())

(x,y,z,w) =  -0.0483099     0.37885   -0.819367 2.77556e-17



That is a pure quaternion, hence encoding a point, and does corresponds to R*p. Magic, is it not? We can prove that the double product of quaternion does corresponds to the rotation. Indeed, a quaternion rather encode an action (a rotation) in $R^4$, but which moves our point p outside of $R^3$. The conjugate rotation brings it back in $R^3$ but applies a second rotation. Since we rotate twice, it is necessary to apply only half of the angle each time.
What if we try to apply the rotation quat on the imaginary part of the quaternion?

In [64]:
qim = Quaternion(quat) # copy
qim[3] = 0
print(qim, quat*qim*quat.conjugate())

((x,y,z,w) = -0.0110012   0.810514  -0.355704          0
, (x,y,z,w) = -0.0110012   0.810514  -0.355704          0
)


What kind of conclusion can we get from this? What geometrical interpretation can we give to $q_{im}$? What about $||q_{im}||$?

## The SLERP example
Let's practice! Implement a linear interpolation between two position p0 and p1, i.e. find the position p(t) with t varying from 0 to 1, with p(0)=p0, p(1)=p1 and continuity between the two extrema.

In [65]:
# %load solution_lerp.py


LERP with quaternions is not working because they are not normalize. Instead we can take either the normalization of the LERP (NLERP), or the spherical LERP (SLERP). 

In [73]:
# %load solution_slerp.py
def _lerp(p0,p1,t):
    return (1-t)*p0+t*p1

def slerp(q0,q1,t):
    assert(t>=0 and t<=1)
    a = AngleAxis(q0.inverse()*q1)
    return Quaternion(AngleAxis(a.angle*t,a.axis))

def nlerp(q0,q1,t):
    q0 = q0.coeffs()
    q1 = q1.coeffs()
    l  = _lerp(q0,q1,t); l/= norm(l)
    return Quaternion(l[3,0],*l[:3].T.tolist()[0])
    
q0 = Quaternion(SE3.Random().rotation)
q1 = Quaternion(SE3.Random().rotation)
gv.applyConfiguration('world/box', [0,0,0]+q0.coeffs().T.tolist()[0])
time.sleep(.1)
for t in np.arange(0,1,.01):
    q = nlerp(q0,q1,t)
    gv.applyConfiguration('world/box', [0,0,0]+q.coeffs().T.tolist()[0])
    gv.refresh()
    time.sleep(.01)
time.sleep(.1)
gv.applyConfiguration('world/box', [0,0,0]+q1.coeffs().T.tolist()[0])


True